In [ ]:
# Connection to Neo4j
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"
password = "testtest"

driver = GraphDatabase.driver(uri, auth=(username, password))

In [ ]:
# Delete all from Neo4j
def delete_all(tx):
    tx.run("MATCH ()-[r]->() DELETE r")
    tx.run("MATCH (n) DELETE n")
           
with driver.session() as session:
    session.execute_write(delete_all)

In [ ]:
distinct_genres = ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'War', 'Musical', 'Documentary', 'IMAX', 'Western', 'Film-Noir', '(no genres listed)']

In [ ]:
# create nodes Genre for movie genres
def create_genre_nodes(tx, name):
    tx.run("MERGE (:Genre {name: $name})", name=name)
    
with driver.session() as session:
    for row in distinct_genres:
        session.execute_write(create_genre_nodes, row)

In [ ]:
# create nodes for Users and Movies and relations :RATED, :BELONGS_TO
ratings_movies = ratings.join(movies, "movieId", "inner")

def create_nodes_and_relationships(tx, userId, movieId, rating, title, genres):
    tx.run("MERGE (u:User {userId: $userId}) "
           "MERGE (m:Movie {movieId: $movieId, title: $title}) "
           "MERGE (u)-[:RATED {rating: $rating}]->(m)",
           userId=userId, movieId=movieId, rating=rating, title=title)
    for genre_name in genres.split("|"):
        tx.run(
            "MATCH (m:Movie {movieId: $movieId}) "
            "MATCH (g:Genre {name: $genre_name}) "
            "MERGE (m)-[:BELONGS_TO]->(g)",
            movieId=movieId, genre_name=genre_name)

with driver.session() as session:
    for row in ratings_movies.collect():
        session.execute_write(create_nodes_and_relationships, row.userId, row.movieId, row.rating, row.title, row.genres)

In [ ]:
# create relations :RECOMMENDED
def create_recommendations(tx, userId, recs):
    for rec in recs:
        tx.run("MATCH (u:User {userId: $userId}), (m:Movie {movieId: $movieId}) "
               "MERGE (u)-[:RECOMMENDED{rating: $rating}]->(m)",
               userId=userId, movieId=rec.movieId, rating=rec.rating)

with driver.session() as session:
    for row in userRecs.collect():
        recs = row.recommendations
        recs = recs if isinstance(recs, list) else [recs]
        session.execute_write(create_recommendations, row.userId, recs)

In [ ]:
# close all
driver.close()